In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.graph_objects as go
%matplotlib inline

In [ ]:
df_1 = pd.read_csv(r'/Users/eliskasimova/Desktop/data_analytics_course_2024/project_folder/labs/second_project/data/clean/df_merged.csv')
df_2 = pd.read_csv(r'/Users/eliskasimova/Desktop/data_analytics_course_2024/project_folder/labs/second_project/data/raw/df_final_web_data_pt_1.txt')
df_3 = pd.read_csv(r'/Users/eliskasimova/Desktop/data_analytics_course_2024/project_folder/labs/second_project/data/raw/df_final_web_data_pt_2.txt')
merged_df = pd.concat([df_2, df_3], axis=0)

df_merged = df_1.merge(merged_df, on='client_id', how='inner')
df_merged['date_time'] = pd.to_datetime(df_merged['date_time'])

### Groups sorting to control/test

In [ ]:
control_group = df_merged[df_merged['variation'] == 'Control']
test_group = df_merged[df_merged['variation'] == 'Test']

# Sort control group
control_group_sorted = control_group.sort_values(by=['client_id', 'visit_id', 'process_step', 'date_time'])

# Sort test group
test_group_sorted = test_group.sort_values(by=['client_id', 'visit_id', 'process_step', 'date_time'])

### Adding age group

In [ ]:
# define age bins and categorize ages
bins = [0, 30, 40, 50, 100] # You can decide the intervals based on your data
labels = ['Under 30', '30-39', '40-49', '50 and above']
df_merged['age_group'] = pd.cut(df_merged['clnt_age'], bins=bins, labels=labels)
control_group_sorted['age_group'] = pd.cut(control_group_sorted['clnt_age'], bins=bins, labels=labels)
test_group_sorted['age_group'] = pd.cut(test_group_sorted['clnt_age'], bins=bins, labels=labels)


### Function to get latest starts

In [ ]:
# Function to get latest starts
def filter_latest_starts(group_df):
    starts_only = group_df[group_df['process_step'] == 'start']
    latest_starts = starts_only.loc[starts_only.groupby('client_id')['date_time'].idxmax()]
    # Merge back to get the full process after the last start
    return df_merged.merge(latest_starts[['client_id', 'date_time']], on=['client_id', 'date_time'], how='inner')

# Apply to both groups
filtered_control = filter_latest_starts(control_group)
filtered_test = filter_latest_starts(test_group)

#Check if it works
client_total_entries = df_merged[df_merged["client_id"] == 2304905]
client_last_start = filtered_control[filtered_control['client_id'] == 2304905 ]

### Completion rate analysis

- number of customers who reached the end of a step out of total who started

In [ ]:
#completion rate based on visit_id
def calculate_within_visit_completion_rate(group):
    # Total unique visits that started
    started_visits = group[group['process_step'] == 'start']['visit_id'].nunique()

    # Unique visits that completed each step
    completed_visits = (
        group[group['process_step'] != 'start']
        .groupby('process_step')['visit_id']
        .nunique()
        .reset_index(name='completed_visits')
    )

    # Add the total started visits as a constant column
    completed_visits['started_visits'] = started_visits

    # Calculate the completion rate
    completed_visits['completion_rate'] = (
        completed_visits['completed_visits'] / completed_visits['started_visits']
    ) * 100

    return completed_visits

# Calculate completion rates for the control and test groups
control_completion_rate = calculate_within_visit_completion_rate(control_group_sorted)
test_completion_rate = calculate_within_visit_completion_rate(test_group_sorted)

# Display the results
print("Control Group Within-Visit Completion Rates:")
display(control_completion_rate)

print("Test Group Within-Visit Completion Rates:")
display(test_completion_rate)

### Completion rate based on client_id 

In [ ]:
#completion rate based on client_id
def calculate_within_visit_completion_rate(group):
    # Total unique visits that started
    started_visits = group[group['process_step'] == 'start']['client_id'].nunique()

    # Unique visits that completed each step
    completed_visits = (
        group[group['process_step'] != 'start']
        .groupby('process_step')['client_id']
        .nunique()
        .reset_index(name='completed_visits')
    )

    # Add the total started visits as a constant column
    completed_visits['started_visits'] = started_visits

    # Calculate the completion rate
    completed_visits['completion_rate'] = (
        completed_visits['completed_visits'] / completed_visits['started_visits']
    ) * 100

    return completed_visits

# Calculate completion rates for the control and test groups
control_completion_rate_id = calculate_within_visit_completion_rate(control_group_sorted)
test_completion_rate_id = calculate_within_visit_completion_rate(test_group_sorted)



# Display the results
print("Control Group Within-Visit Completion Rates:")
display(control_completion_rate_id)

print("Test Group Within-Visit Completion Rates:")
display(test_completion_rate_id)

### Completion rate chart

In [ ]:
merged_data = pd.merge(control_completion_rate, test_completion_rate, on='process_step', suffixes=('_control', '_test'))

# Plotting
plt.figure(figsize=(10, 6))
bar_width = 0.35
index = range(len(merged_data))

# Create bar chart
plt.bar(index, merged_data['completion_rate_control'], bar_width, label='Control Group', color='blue')
plt.bar(
    [i + bar_width for i in index],
    merged_data['completion_rate_test'],
    bar_width,
    label='Test Group',
    color='orange'
)

# Add labels, title, and legend
plt.xlabel('Process Step')
plt.ylabel('Completion Rate (%)')
plt.title('Completion Rates by Process Step')
plt.xticks([i + bar_width / 2 for i in index], merged_data['process_step'])
plt.legend()
plt.tight_layout()

# Show the plot
plt.show()

### Completion rates for the control and test groups based on age group

In [ ]:
def calculate_within_visit_completion_rate_by_age(group):
    # Total unique visits that started
    started_visits = group[group['process_step'] == 'start']['client_id'].nunique()

    # Unique visits that completed each step, grouped by age_group
    completed_visits = (
        group[group['process_step'] != 'start']
        .groupby(['process_step', 'age_group'])['client_id']
        .nunique()
        .reset_index(name='completed_visits')
    )

    # Add the total started visits as a constant column
    completed_visits['started_visits'] = started_visits

    # Calculate the completion rate
    completed_visits['completion_rate'] = (
        completed_visits['completed_visits'] / completed_visits['started_visits']
    ) * 100

    return completed_visits

# Calculate completion rates for the control and test groups based on age group
control_completion_rate_by_age = calculate_within_visit_completion_rate_by_age(control_group_sorted)
test_completion_rate_by_age = calculate_within_visit_completion_rate_by_age(test_group_sorted)

# Display the results for control and test groups
print("Control Group Completion Rates by Age Group:")
display(control_completion_rate_by_age)

print("Test Group Completion Rates by Age Group:")
display(test_completion_rate_by_age)


### Completion rate hypothesis test using two-proportion z-test

In [ ]:
import scipy.stats as stats

# function to perform a two-proportion z-test
def two_proportion_z_test(p1, p2, n1, n2):
    # Calculate the pooled proportion
    P = (p1 * n1 + p2 * n2) / (n1 + n2)
    
    # Calculate the standard error
    SE = (P * (1 - P) * (1 / n1 + 1 / n2)) ** 0.5
    
    # Calculate the z-statistic,
    """
    The Z-statistic measures the difference between control and test in terms of standard deviations. 
    The higher the z-stat it is(whether positive or negative), the more significant the difference between the groups. 
    If the p-value is less than the chosen significance level (e.g., 0.05), we reject the null hypothesis 
    and conclude that the observed difference is statistically significant.
    """
    z = (p1 - p2) / SE
    
    # Calculate the p-value
    p_value = 2 * (1 - stats.norm.cdf(abs(z)))  # Two-tailed test
    
    return z, p_value

# Hypothesis testing for each step
steps = ['confirm', 'step_1', 'step_2', 'step_3']

for step in steps:
s    # Get completion rates for both control and test groups for the current step
    control_completions = control_completion_rate[control_completion_rate['process_step'] == step]['completion_rate'].values[0]
    test_completions = test_completion_rate[test_completion_rate['process_step'] == step]['completion_rate'].values[0]
    
    control_total = control_completion_rate[control_completion_rate['process_step'] == step]['started_visits'].values[0]
    test_total = test_completion_rate[test_completion_rate['process_step'] == step]['started_visits'].values[0]

    # Calculate proportions (completion rate)
    p_control = control_completions / 100  # Convert completion rate to a proportion
    p_test = test_completions / 100  # Convert completion rate to a proportion
    
    # Perform the two-proportion z-test
    z_stat, p_value = two_proportion_z_test(p_control, p_test, control_total, test_total)
    
    # Print results
    print(f"Step: {step}")
    print(f"Z-statistic: {z_stat:.4f}")
    print(f"P-value: {p_value:.4f}")
    
    # Interpretation based on p-value
    if p_value < 0.05:
        print(f"There is a significant difference in completion rates between control and test group for step: {step}.")
    else:
        print(f"There is no significant difference in completion rates between control and test group for step: {step}.")
    print("\n")


### Completion Rate with a Cost-Effectiveness Threshold

In [ ]:
# calculate the completion rate for each group across all steps
control_mean = control_completion_rate['completion_rate'].mean()
test_mean = test_completion_rate['completion_rate'].mean()

# two-sample t-test to check if the completion rates are significantly different
_, p_value = st.ttest_ind(control_completion_rate['completion_rate'], 
                                test_completion_rate['completion_rate'], 
                                alternative='two-sided')

# display the results for completion rate difference significance
print(f"Average completion rate per Control group: {control_mean:.2f}%")
print(f"Average completion rate per Test group: {test_mean:.2f}%")
print(f"Statistic: {_:.4f}") #how much the means of the two groups differ relative to the variability of the data
print(f"P-value: {p_value:.4f}") #check if the difference in average tenure between the two groups is statistically significant.

# hypothesis test: Is the completion rate difference significant?
alpha = 0.05  
if p_value < alpha:
    print("Reject the null hypothesis: The completion rates are significantly different between the Test and Control groups.")
else:
    print("Fail to reject the null hypothesis: The completion rates are not significantly different between the Test and Control groups.")

# check if the increase in completion rate meets the 5% threshold
completion_rate_increase = test_mean - control_mean
print(f"Completion rate increase: {completion_rate_increase:.2f}%")

# check if the increase meets or exceeds the 5% threshold
if completion_rate_increase >= 5:
    print("The completion rate increase meets the 5% threshold, justifying the cost of the new design.")
else:
    print("The completion rate increase does not meet the 5% threshold. The new design may not justify its cost.")


### Additional hypothesis test 

 Tenure: Test if client tenure (how long they've been with Vanguard) influences engagement with the new design, using two-sample t-test

In [ ]:
control_tenure = control_group_sorted['clnt_tenure_yr'] 
test_tenure = test_group_sorted['clnt_tenure_yr']        
# two-sample t-test to compare the means of tenure between the two groups
_, p_value = st.ttest_ind(control_tenure, test_tenure, equal_var=True)  # assuming equal variance
#results
print(f"Average tenure per Control group: {control_tenure.mean():.2f} years")
print(f"Average tenure per Test group: {test_tenure.mean():.2f} years")
print(f"Statistic: {_:.4f}")
print(f"P-value: {p_value:.4f}")

# hypothesis test: Is there a significant difference in tenure between the two groups?
alpha = 0.05  # significance level

if p_value < alpha:
    print("Reject the null hypothesis: The average tenure is significantly different between the Test and Control groups.")
else:
    print("Fail to reject the null hypothesis: The average tenure is not significantly different between the Test and Control groups.")


#### Age: Test if the average age of clients engaging with the new design is different from those using the old design

In [ ]:
control_age = control_group_sorted['clnt_age']
test_age = test_group_sorted['clnt_age']

# two-sample t-test to compare the means of age between the two groups
_, p_value = st.ttest_ind(control_age, test_age, equal_var=True)  # Assuming equal variance

# results
print(f"Control Group Mean Age: {control_age.mean():.2f} years")
print(f"Test Group Mean Age: {test_age.mean():.2f} years")
print(f"Statistic: {_:.4f}")
print(f"P-value: {p_value:.4f}")

# hypothesis test: Is there a significant difference in age between the two groups?
alpha = 0.05  #

if p_value < alpha:
    print("Reject the null hypothesis: The average age is significantly different between the Test and Control groups.")
else:
    print("Fail to reject the null hypothesis: The average age is not significantly different between the Test and Control groups.")

The statistical test (t-test) suggests that the average ages of the two groups are indeed different. Since the p-value is less than 0.05, this indicates a statistically significant difference in age between the Test and Control groups.